In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
# load all necessary libraries
import pandas as pd
import numpy as np # linear algebra

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection  import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


from gensim.models.word2vec import Word2Vec
from gensim import models
from gensim.models import KeyedVectors


from sklearn.linear_model import LogisticRegression
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D,Lambda,BatchNormalization,Activation
from keras.layers import Conv1D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
import keras.backend as K

import tensorflow as tf
from keras.preprocessing import sequence

from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot
from keras.layers import Input,MaxPool1D
from keras.models import Model


from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
Using TensorFlow backend.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8",

In [2]:
## initialise the inbuilt Stemmer and the Lemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def preprocess(document, stem=True):
    'changes document to lower case, removes stopwords and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

In [3]:
seq_len = max_len = 200 #512
b_size = 128
n_chan = 10
f_size = 3 # filter size


In [4]:
glove_input_file = './../glove.6B.200d.txt'
word2vec_output_file = 'glove.6B.200d.w2vformat.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
glove_model = KeyedVectors.load_word2vec_format("glove.6B.200d.w2vformat.txt", binary=False)

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
def sent_vec(sent):
    wv_res = np.zeros(glove_model.vector_size)
    ctr = 1
    for w in sent:
        if w in glove_model:
            ctr += 1
            wv_res += glove_model[w]
    wv_res = wv_res/ctr
    #return (wv_res, ctr)
    return wv_res

In [6]:

data = pd.read_csv("./../deceptive-opinion.csv")
data = data.loc[:,['text','deceptive']]

# stem messages
#messages = [preprocess(message, stem=True) for message in data.text]
data.text = data.text.apply(lambda message : preprocess(message, stem=False))
data['deceptive'] = data.deceptive.map({'truthful':1, 'deceptive':0})

# X = data.text
# y = data.deceptive
#X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.30,random_state=1)


df_train, df_test= train_test_split(data,  test_size = 0.20,random_state=1)
df_truthful = df_train.loc[df_train.deceptive == 1,:]
X_train = df_truthful.text
X_test=df_test.text
y_train= df_truthful.deceptive
y_test=df_test.deceptive

print('Training set size : ', (X_train.shape[0]))
print('Test set size : ', (X_test.shape[0]))



Training set size :  649
Test set size :  320


In [7]:
X, y = [], []

for idx,row in df_truthful.iterrows():
    label, text = row['deceptive'],row['text']
    X.append(text.split())
    y.append(label)
X, y = np.array(X), np.array(y)

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


In [8]:
X_test, y_test = [], []
for idx,row in df_test.iterrows():
    label, text = row['deceptive'],row['text']
    X_test.append(text.split())
    y_test.append(label)
X_test, y_test = np.array(X_test), np.array(y_test)

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [9]:
stop_words = stopwords.words("english")
train_doc_vecs = pd.DataFrame()
for doc in X:    
    doc_words = [term for term in doc if term not in stop_words]
    b = sent_vec(doc_words)
    np_array = np.array(b.tolist())
    reshaped_array = np.reshape(np_array, (1,200))
    a_dataframe = pd.DataFrame(reshaped_array)
    train_doc_vecs = train_doc_vecs.append(a_dataframe)

    
test_doc_vecs = pd.DataFrame()
for doc in X_test:    
    doc_words = [term for term in doc if term not in stop_words]
    b = sent_vec(doc_words)
    np_array = np.array(b.tolist())
    reshaped_array = np.reshape(np_array, (1,200))
    a_dataframe = pd.DataFrame(reshaped_array)
    test_doc_vecs = test_doc_vecs.append(a_dataframe)

In [10]:
vocab_size = train_doc_vecs.shape[1]

In [11]:
print('Pad sequences (samples x time)')
x_train =  train_doc_vecs.values#sequence.pad_sequences(train_doc_vecs.toarray(), maxlen=max_len)
x_test =  test_doc_vecs.values#sequence.pad_sequences(test_doc_vecs.toarray(), maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (649, 200)
x_test shape: (320, 200)


In [12]:

def define_discriminator(in_shape=(max_len,1)):

    D = Sequential()
    D.add(Conv1D(n_chan,f_size,activation='relu',input_shape = (seq_len,1)))
    D.add(Flatten())
    D.add(Dense(1, activation='sigmoid'))
    D.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    return D



In [13]:
# define the standalone generator model
def define_generator(latent_dim):
    def Conv1DTranspose(inp,nf,ks,s=2,p='same'):
        x1 = Lambda(lambda x : K.expand_dims(x,axis=2))(inp)
        x2 = Conv2DTranspose(filters=nf,kernel_size=(ks,1),strides=(s,1),padding=p)(x1)
        return Lambda(lambda x :K.squeeze(x,axis=2))(x2)


    G = Sequential()
    G.add(Dense(int(seq_len/8)*n_chan,input_shape=(latent_dim,)))
    G.add(Reshape((int(seq_len/8),n_chan)))
    G.add(BatchNormalization(momentum= 0.8,epsilon=1.e-5))
    for i in range(0,2):
        G.add(Lambda(lambda x : Conv1DTranspose(x,n_chan,f_size)))
        G.add(BatchNormalization(momentum= 0.8,epsilon=1.e-5))

    G.add(Lambda(lambda x : Conv1DTranspose(x,1,3)))
    G.add(Activation('sigmoid'))
    G.summary()
    return G



In [14]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [15]:
# load data
def load_real_samples():
    # load dataset
    (trainX, _), (_, _) = (x_train,y_train),(x_test,y_test)#load_data()

    return trainX

In [16]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select data
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

In [17]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

In [18]:
def plot_history(d1_hist, d2_hist, g_hist):
    # plot history
    pyplot.title("GAN+GLOVE")
    pyplot.plot(d1_hist, label='disc_real')
    pyplot.plot(d2_hist, label='disc_fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    pyplot.savefig('GAN_GLOVE_line_plot_loss.png')
    pyplot.close()

In [19]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    c1_hist, c2_hist, g_hist = list(), list(), list()
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            c1_hist.append(d_loss1)
            c2_hist.append(d_loss2)
            g_hist.append(g_loss)
            
            #print(X_real.shape,X_fake.shape)
            
    # save the generator model
    #g_model.save('generator.h5')
    #d_model.save('dis_generator.h5')
    plot_history(c1_hist, c2_hist, g_hist)
    return g_model,d_model

In [20]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load data
dataset = load_real_samples()

dataset = np.reshape(dataset,(dataset.shape[0],dataset.shape[1],1))

# train model
model,d_model= train(generator, discriminator, gan_model, dataset, latent_dim)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 250)               25250     
_________________________________________________________________
reshape_1 (Reshape)          (None, 25, 10)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 10)            40        
_________________________________________________________________
lambda_1 (Lambda)            (None, 50, 10)            0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 10)            40        
_________________________________________________________________
lambda_6 (Lambda)            (None, 100, 10)           0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 100, 10)           40        
__________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 1/5, d1=0.705, d2=0.841 g=0.621
>1, 2/5, d1=0.677, d2=0.744 g=0.704
>1, 3/5, d1=0.664, d2=0.636 g=0.821
>1, 4/5, d1=0.651, d2=0.527 g=0.958
>1, 5/5, d1=0.643, d2=0.454 g=1.099
>2, 1/5, d1=0.631, d2=0.373 g=1.257
>2, 2/5, d1=0.619, d2=0.305 g=1.385
>2, 3/5, d1=0.603, d2=0.260 g=1.567
>2, 4/5, d1=0.591, d2=0.217 g=1.744
>2, 5/5, d1=0.574, d2=0.179 g=1.875
>3, 1/5, d1=0.555, d2=0.152 g=2.030
>3, 2/5, d1=0.534, d2=0.129 g=2.167
>3, 3/5, d1=0.516, d2=0.112 g=2.310
>3, 4/5, d1=0.487, d2=0.100 g=2.423
>3, 5/5, d1=0.464, d2=0.085 g=2.531
>4, 1/5, d1=0.443, d2=0.074 g=2.654
>4, 2/5, d1=0.411, d2=0.071 g=2.762
>4, 3/5, d1=0.385, d2=0.062 g=2.845
>4, 4/5, d1=0.364, d2=0.057 g=2.906
>4, 5/5, d1=0.339, d2=0.053 g=2.971
>5, 1/5, d1=0.316, d2=0.051 g=3.029
>5, 2/5, d1=0.286, d2=0.046 g=3.125
>5, 3/5, d1=0.263, d2=0.044 g=3.172
>5, 4/5, d1=0.245, d2=0.040 g=3.221
>5, 5/5, d1=0.224, d2=0.040 g=3.268
>6, 1/5, d1=0.200, d2=0.037 g=3.304
>6, 2/5, d1=0.182, d2=0.036 g=3.361
>6, 3/5, d1=0.169, d2=0.034 